# Prepare_AO_LR
This notebook prepare anatomical ontology text file for left and right side of the brain by postfixing "L" or "R" for ROI name. In the case of right side, ROI number was modified by adding a constant.

- input
    - 1_VC_pruned_segmented_VC_combined_gene_VC_fiber_VC.json
- output
    - R.json
%     - 1_VC_pruned_segmented_VC_combined_gene_VC_fiber_VC_RL.json

In [24]:
dir_data = 'data'

# for detailed ROI
fn_input_AO = 'AObase_d_g_f.json'

In [ ]:
id_spacer = 2*10**9 # this number is added to ID in the right hemisphere of annotation volume
fn_output_AO_R = 'AO_R.json'
fn_output_AO_L = 'AO_L.json'

In [23]:
import os
import nrrd
import numpy as np
import pandas as pd
import json
import copy
from collections import OrderedDict
from jsonpath_rw import jsonpath, parse

# Load data

In [25]:
with open(os.path.join(dir_data, fn_input_AO)) as f:
    df_ori = json.load(f, object_pairs_hook=OrderedDict)

# Add ID-spacer to distinguish ROI in the right hemisphere

In [26]:
def Add_IDspacer(match_id, match_fullpath):
    if match_id == 0: return # Children is not defined for a root node.
    # id_offset = 10**9 # ID of segmented node = original ID + this offset
    global df_R
    global id_spacer
    global df_L
    print(match_fullpath)
    print(match_id)
    ## right side
    source_acronym =  eval("df_R"+\
            str(match_fullpath).replace('.','')\
            .replace('msg', "['msg']").replace('children', "['children']")\
            .replace('id', "['acronym']")) # acronym for matched ID
    source_name =  eval("df_R"+\
            str(match_fullpath).replace('.','')\
            .replace('msg', "['msg']").replace('children', "['children']")\
            .replace('id', "['name']")) # name for matched ID
    source_path = "df_R"+str(match_fullpath)\
        [0:str(match_fullpath).rfind("]")+1].replace('.','')\
        .replace('msg', "['msg']")\
        .replace('children', "['children']")
    parent_id = eval("df_R"+\
                    str(match_fullpath).replace('.','')\
                    .replace('msg', "['msg']").replace('children',"['children']")\
                    .replace('id', "['parent_structure_id']"))
    exec(source_path+"['id'] = "+str(match_id + id_spacer))
    exec(source_path+"['acronym'] = '"+source_acronym+"_R'")
    exec(source_path+'["name"] = "'+source_name+'_R"') # Note Barrington's nucleus
    if parent_id is not None:
        exec(source_path+"['parent_structure_id'] = "+str(parent_id + id_spacer))
    
    ## left side
    source_acronym =  eval("df_L"+\
            str(match_fullpath).replace('.','')\
            .replace('msg', "['msg']").replace('children', "['children']")\
            .replace('id', "['acronym']")) # acronym for matched ID
    source_name =  eval("df_L"+\
            str(match_fullpath).replace('.','')\
            .replace('msg', "['msg']").replace('children', "['children']")\
            .replace('id', "['name']")) # name for matched ID
    source_path = "df_L"+str(match_fullpath)\
        [0:str(match_fullpath).rfind("]")+1].replace('.','')\
        .replace('msg', "['msg']")\
        .replace('children', "['children']")
    exec(source_path+"['acronym'] = '"+source_acronym+"_L'")
    exec(source_path+'["name"] = "'+source_name+'_L"') # Note Barrington's nucleus
    
    return [match_id, source_acronym, source_name] # Leafed ID and its acronym

In [27]:
jsonpath_expr = parse('$..id')
df_R = copy.deepcopy(df_ori)
df_L = copy.deepcopy(df_ori)

temp_list = [Add_IDspacer(match.value, match.full_path)\
            for match in reversed(jsonpath_expr.find(df_R))]
# temp_list = []
# temp_list = [Add_IDspacer(match.value, match.full_path)\
#             for match in reversed(jsonpath_expr.find(df_R))]
# ID_R = np.array(temp_list)

msg.[1].id
1000000028
msg.[0].children.[3].id
73
msg.[0].children.[2].id
1000000012
msg.[0].children.[1].id
1009
msg.[0].children.[0].children.[3].id
1000000013
msg.[0].children.[0].children.[2].children.[1].children.[3].id
589508455
msg.[0].children.[0].children.[2].children.[1].children.[2].id
846
msg.[0].children.[0].children.[2].children.[1].children.[1].id
91
msg.[0].children.[0].children.[2].children.[1].children.[0].id
989
msg.[0].children.[0].children.[2].children.[1].id
519
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[5].id
1049
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[4].id
1041
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[3].id
1033
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[2].id
1025
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[1].children.[1].id
1064
msg.[0].children.[0].children.[2].children.[0].children.[1].children.[1].children.[0].id
1056


msg.[0].children.[0].children.[0].children.[0].children.[0].children.[0].children.[10].children.[0].children.[1].id
412
msg.[0].children.[0].children.[0].children.[0].children.[0].children.[0].children.[10].children.[0].children.[0].id
448
msg.[0].children.[0].children.[0].children.[0].children.[0].children.[0].children.[10].children.[0].id
723
msg.[0].children.[0].children.[0].children.[0].children.[0].children.[0].children.[10].id
714
msg.[0].children.[0].children.[0].children.[0].children.[0].children.[0].children.[9].children.[4].id
1081
msg.[0].children.[0].children.[0].children.[0].children.[0].children.[0].children.[9].children.[3].id
1054
msg.[0].children.[0].children.[0].children.[0].children.[0].children.[0].children.[9].children.[2].id
827
msg.[0].children.[0].children.[0].children.[0].children.[0].children.[0].children.[9].children.[1].id
556
msg.[0].children.[0].children.[0].children.[0].children.[0].children.[0].children.[9].children.[0].id
707
msg.[0].children.[0].childr

# Save 

In [28]:
with open(os.path.join(dir_data, fn_output_AO_R), mode='w') as fw:
    json.dump(df_R, fw, indent=4)
with open(os.path.join(dir_data, fn_output_AO_L), mode='w') as fw2:
    json.dump(df_L, fw2, indent=4)

# Check data

In [29]:
df_ori

OrderedDict([('success', True),
             ('id', 0),
             ('start_row', 0),
             ('num_rows', 1),
             ('total_rows', 1),
             ('msg',
              [OrderedDict([('id', 997),
                            ('atlas_id', -1),
                            ('ontology_id', 1),
                            ('acronym', 'root'),
                            ('name', 'root'),
                            ('color_hex_triplet', 'FFFFFF'),
                            ('graph_order', 0),
                            ('st_level', None),
                            ('hemisphere_id', 3),
                            ('parent_structure_id', None),
                            ('children',
                             [OrderedDict([('id', 8),
                                           ('atlas_id', 0),
                                           ('ontology_id', 1),
                                           ('acronym', 'grey'),
                                           ('name',
